In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
import json
from tensorflow.keras.utils import to_categorical   
from sklearn.model_selection import train_test_split    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense         
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau       
def collect_and_train(gesture_name, data_path, model_path, secs_for_action=15):
    """
    사용자로부터 제스처 데이터를 수집하고 기존 제스처 파일들과 함께 모델을 학습합니다.
    """

    actions = []  
    seq_length = 30    

    # MediaPipe hands model
    mp_hands = mp.solutions.hands      
    mp_drawing = mp.solutions.drawing_utils       
    hands = mp_hands.Hands(                     
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(0)

    created_time = int(time.time())             
    os.makedirs(data_path, exist_ok=True)         

    # 새로운 데이터 수집
    while cap.isOpened():
        for idx, action in enumerate([gesture_name]): 
            data = []

            ret, img = cap.read()                
            img = cv2.flip(img, 1)          

            cv2.putText(img, f'Collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
            cv2.imshow('img', img)       
            cv2.waitKey(3000)

            start_time = time.time()      

            while time.time() - start_time < secs_for_action:   
                ret, img = cap.read()                               
                img = cv2.flip(img, 1)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
                result = hands.process(img)                       
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                if result.multi_hand_landmarks is not None:      
                    for res in result.multi_hand_landmarks:       
                        joint = np.zeros((21, 4))               
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z, lm.visibility]    

                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]      
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]    
                        v = v2 - v1      
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]   

                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))         

                        angle = np.degrees(angle)         

                        angle_label = np.array([angle], dtype=np.float32)           
                        angle_label = np.append(angle_label, idx)                  

                        d = np.concatenate([joint.flatten(), angle_label])          
                        data.append(d)                                              
                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)      

                cv2.imshow('img', img)              
                if cv2.waitKey(1) == ord('q'):
                    return

            data = np.array(data)
            print(action, data.shape)
            np.save(os.path.join(data_path, f'raw_{action}_{created_time}'), data)

            full_seq_data = []
            for seq in range(len(data) - seq_length):
                full_seq_data.append(data[seq:seq + seq_length])

            full_seq_data = np.array(full_seq_data)
            print(action, full_seq_data.shape)
            np.save(os.path.join(data_path, f'seq_{action}_{created_time}'), full_seq_data)
        break
    cap.release()
    cv2.destroyAllWindows()

    # 기존 데이터 로드 및 병합
    x_data = []
    y_data = []

    for filename in os.listdir(data_path):
        if filename.startswith('seq_') and filename.endswith('.npy'):
            action_name = filename.split('_')[1]
            if action_name not in actions:
                actions.append(action_name)

            action_index = actions.index(action_name)
            data = np.load(os.path.join(data_path, filename))
            x_data.extend(data[:, :, :-1].tolist())
            y_data.extend([action_index] * len(data))

    x_data = np.array(x_data)
    y_data = np.array(y_data)
    y_data = to_categorical(y_data, num_classes=len(actions))

    x_data = x_data.astype(np.float32)
    y_data = y_data.astype(np.float32)

    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=43)

    model = Sequential([
        LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
        Dense(32, activation='relu'),
        Dense(len(actions), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    history = model.fit(
        x_train,
        y_train,
        validation_data=(x_val, y_val),
        epochs=200,
        callbacks=[
            ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
            ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
        ]
    )

    # 모델 학습 후 레이블 저장
    with open('ai/training/data/gesture_labels.json', 'w') as f:
        json.dump(actions, f)

if __name__ == "__main__":
    gesture_name = input("학습할 제스처 이름을 입력하세요: ")
    data_path = 'ai/training/data/gesture_seed_data'
    model_path = 'ai/models/gesture_model.h5'

    collect_and_train(gesture_name, data_path, model_path)

2025-03-27 13:54:38.888257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743051278.906376   21445 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743051278.911550   21445 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743051278.926766   21445 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743051278.926784   21445 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743051278.926785   21445 computation_placer.cc:177] computation placer alr

boss (142, 100)
boss (112, 30, 100)


I0000 00:00:1743051306.162529   21445 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4225 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/uk/dp_pj/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200


I0000 00:00:1743051309.046334   21566 service.cc:152] XLA service 0x16800710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743051309.046353   21566 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-03-27 13:55:09.107440: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743051309.454529   21566 cuda_dnn.cc:529] Loaded cuDNN version 90800
2025-03-27 13:55:11.110839: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_26', 20 bytes spill stores, 20 bytes spill loads



...

143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - acc: 0.4484 - loss: 23.7116 - val_acc: 0.6884 - val_loss: 3.7038 - learning_rate: 0.0010
Epoch 5/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7526 - loss: 9.4851
Epoch 5: val_acc improved from 0.68836 to 0.80868, saving model to models/gesture.h5


143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.7527 - loss: 9.4695 - val_acc: 0.8087 - val_loss: 2.6586 - learning_rate: 0.0010
Epoch 6/200
135/143 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7737 - loss: 4.5412
Epoch 6: val_acc improved from 0.80868 to 0.83629, saving model to models/gesture.h5


143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - acc: 0.7747 - loss: 4.5215 - val_acc: 0.8363 - val_loss: 3.8625 - learning_rate: 0.0010
Epoch 7/200
134/143 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7062 - loss: 34.0988
Epoch 7: val_acc did not improve from 0.83629
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.7018 - loss: 33.7754 - val_acc: 0.7357 - val_loss: 9.8977 - learning_rate: 0.0010
Epoch 8/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.6605 - loss: 14.5617
Epoch 8: val_acc did not improve from 0.83629
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - acc: 0.6608 - loss: 14.5190 - val_acc: 0.5858 - val_loss: 6.2575 - learning_rate: 0.0010
Epoch 9/200
135/143 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.6469 - loss: 9.4157
Epoch 9: val_acc improved from 0.83629 to 0.86193, saving model to models/gesture.h5


143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - acc: 0.6514 - loss: 9.3077 - val_acc: 0.8619 - val_loss: 2.1387 - learning_rate: 0.0010
Epoch 10/200
142/143 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.8434 - loss: 2.0944
Epoch 10: val_acc did not improve from 0.86193
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - acc: 0.8429 - loss: 2.1016 - val_acc: 0.7673 - val_loss: 2.2248 - learning_rate: 0.0010
Epoch 11/200
134/143 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.5840 - loss: 16.5925
Epoch 11: val_acc did not improve from 0.86193
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - acc: 0.5729 - loss: 17.4968 - val_acc: 0.5049 - val_loss: 18.8964 - learning_rate: 0.0010
Epoch 12/200
137/143 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.4618 - loss: 29.8234
Epoch 12: val_acc did not improve from 0.86193
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc: 0.4604 - loss: 29.8285 - val_acc: 0.0888 - val_loss: 306.5481 - learning_rate: 0.0010
Epoch 13/200
140/143 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.1419 - loss: 88.

KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import json

def recognize_gesture(model_path):
    """
    실시간으로 제스처를 인식하고 화면에 표시합니다.
    """

    seq_length = 30

    try:
        model = load_model(model_path)
    except FileNotFoundError:
        print(f"Error: Model file not found at {model_path}")
        return

    # 모델의 레이블(제스처 이름) 가져오기
    try:
        with open('models/gesture_labels.json', 'r') as f:
            actions = json.load(f)
    except FileNotFoundError:
        print("Error: gesture_labels.json file not found.")
        return

    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    seq = []
    action_seq = []

    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            break

        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)
        img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                joint = np.zeros((21, 4))
                for j, landmark in enumerate(hand_landmarks.landmark):
                    joint[j] = [landmark.x, landmark.y, landmark.z, landmark.visibility]

                v1 = joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19], :3]
                v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], :3]
                v = v2 - v1
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18], :],
                    v[[1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19], :]))
                angle = np.degrees(angle)

                d = np.concatenate([joint.flatten(), angle])
                seq.append(d)

                mp_drawing.draw_landmarks(img_bgr, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                if len(seq) < seq_length:
                    continue

                input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
                y_pred = model.predict(input_data).squeeze()

                if y_pred.ndim == 0:
                    print("No prediction result.")
                    continue

                i_pred = int(np.argmax(y_pred))
                conf = y_pred[i_pred]

                if conf < 0.9:
                    continue

                predicted_action = actions[i_pred]
                action_seq.append(predicted_action)

                if len(action_seq) < 3:
                    continue

                this_action = '?'
                if action_seq[-1] == action_seq[-2] == action_seq[-3]:
                    this_action = predicted_action

                # 제스처 이름 및 신뢰도를 화면에 표시
                text = f'{this_action.upper()} ({conf:.2f})'
                cv2.putText(img_bgr, text, org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

        cv2.imshow('img', img_bgr)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    model_path = 'models/gesture_model.h5'
    recognize_gesture(model_path)

Error: Model file not found at models/model.h5
